In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
from aerb_unv_reader import UNV

In [3]:
from material_prop_reader import material_section_read

In [4]:
with open('EPR_mat.pickle', 'rb') as handle:
    EPR_mat = pickle.load(handle)
with open('GROUPS.pickle', 'rb') as handle:
    GROUPS = pickle.load(handle)
with open('ELEM_GROUPS.pickle', 'rb') as handle:
    ELEM_GROUPS = pickle.load(handle)
with open('NODE_GROUPS.pickle', 'rb') as handle:
    NODE_GROUPS = pickle.load(handle)

In [5]:
EPR_geo = UNV('InputsV2/PHASE_3_STAGE_1_INPUT_v2/02_Model/CRUAS_NPP.unv')

# Material Properties

In [6]:
MATERIALS = {}
for key, mat in EPR_mat.material.items():
    MATERIALS[key] = mat

In [7]:
with open('MATERIALS.pickle', 'wb') as handle:
    pickle.dump(MATERIALS, handle, protocol=2)

# Finding Unique MATERIALS PPTs

In [8]:
Unique_MATERIALS = {}
Unique_MATERIALS_index = {}
def check_unique_mat(mat, Unique_MATERIALS):
    for key,matChar in Unique_MATERIALS.items():
        if abs(matChar['E']-mat['E'])>100:
            continue
        elif abs(matChar['RHO']-mat['RHO'])>1:
            continue
        elif abs(matChar['NU']-mat['NU'])>0.05:
            continue
        else:
            return key
for key, matChar in MATERIALS.items():
    unq = check_unique_mat(matChar, Unique_MATERIALS)
    if unq:
        Unique_MATERIALS_index[key] = unq
    else:
        Unique_MATERIALS[key] = matChar
        Unique_MATERIALS[key]['groups'] = []
        Unique_MATERIALS_index[key] = key

In [9]:
MATERIAL_ASSIGNMENTS = {}
for grp, matName in EPR_mat.material_assignment.items():
    if matName in MATERIAL_ASSIGNMENTS.keys():
        MATERIAL_ASSIGNMENTS[matName].append(grp)
    else:
        MATERIAL_ASSIGNMENTS[matName] = [grp]
    Unique_MATERIALS[Unique_MATERIALS_index[matName]]['groups'].append(grp)

In [10]:
with open('Unique_MATERIALS.pickle', 'wb') as handle:
    pickle.dump(Unique_MATERIALS, handle, protocol=2)

## Check Whether All Ifo Are PASSED

In [11]:
MATERIAL_ASSIGNMENTS

{'m1': ['BD4_V8',
  'BD4_V9',
  'BD4_V10',
  'BD4_V11',
  'BD4_V12',
  'BD4_V13',
  'BD4_V14',
  'BD4_V15'],
 'N_PESANT': ['BD4_V1',
  'BD4_V2',
  'BD4_V3',
  'BD4_V4',
  'BD4_V5',
  'BD4_V6',
  'BD4_V7',
  'BD5_VOILES',
  'BD6_VOILES',
  'BD1_VOILE',
  'BD2_VOILE',
  'BD3_VOILES',
  'BD7_VOILES'],
 'm2': ['BD5_PTR', 'BD6_PTR'],
 'M_RAD_I': ['RAD_INFD', 'RAD_INFG', 'RAD_INFR'],
 'M_PB': ['PO_BRI8',
  'PO_BRI7',
  'PO_BRI6',
  'PO_BRI5',
  'PO_BRI4',
  'PO_BRI3',
  'PO_BRI2',
  'PO_BRI1',
  'PO_BRI29',
  'PO_BRI28',
  'PO_BRI27',
  'PO_BRI26',
  'PO_BRI25',
  'PO_BRI24',
  'PO_BRI23',
  'PO_BRI22',
  'PO_BRI21',
  'PO_BRI20',
  'PO_BRI19',
  'PO_BRI18',
  'PO_BRI17',
  'PO_BRI16',
  'PO_BRI15',
  'PO_BRI14',
  'PO_BRI13',
  'PO_BRI12',
  'PO_BRI11',
  'PO_BRI10',
  'PO_BRI9',
  'PO_BRI32',
  'PO_BRI31',
  'PO_BRI30',
  'PO_BRI39',
  'PO_BRI38',
  'PO_BRI37',
  'PO_BRI36',
  'PO_BRI35',
  'PO_BRI34',
  'PO_BRI33',
  'PO_BD4_8',
  'PO_BD4_7',
  'PO_BD4_6',
  'PO_BD4_5',
  'PO_BD4_4',
  'P

In [12]:
## READ ALL ELEMENTS
ELEMENTS = {}
for key,value in EPR_geo.fem['elements'][21].items():
    ELEMENTS[key] = {'type':21, 'connectivity':value[-2:]}
for key,value in EPR_geo.fem['elements'][91].items():
    ELEMENTS[key] = {'type':91, 'connectivity':value}
for key,value in EPR_geo.fem['elements'][94].items():
    ELEMENTS[key] = {'type':94, 'connectivity':value}
Unique_MATERIALS_IDX = pd.DataFrame(data=None, index=list(ELEMENTS.keys()), columns=['MATERIALS'])
Unique_MATERIALS_IDX2 = pd.DataFrame(data=None, index=list(ELEMENTS.keys()), columns=['MATERIALS'])
for matName, matDict in Unique_MATERIALS.items():
    for grp in matDict['groups']:
        for eid in GROUPS[grp]['group']:
            Unique_MATERIALS_IDX.loc[eid,'MATERIALS'] = matName
for key, matChar in MATERIAL_ASSIGNMENTS.items():
    for grp in matChar:
        for eid in GROUPS[grp]['group']:
            Unique_MATERIALS_IDX2.loc[eid,'MATERIALS'] = key

In [ ]:
Unique_MATERIALS_IDX[Unique_MATERIALS_IDX.isnull().any(axis=1)]

In [ ]:
Unique_MATERIALS_IDX2[Unique_MATERIALS_IDX2.isnull().any(axis=1)]

In [ ]:
list(ELEMENTS.keys())[-1] + list(ELEMENTS.keys())[0]

In [15]:
len(EPR_mat.material)

479

In [18]:
len(Unique_MATERIALS)

215